### **Calculando a velocidade espacial em coordenadas galactocentricas usando APOGEE e GAIADR2**
Fredi Quispe H.

Para calcular a velocidade é necessario ter os seguintes parâmetros:

    1 Posição (ascensão recta e declinação)
    2 Movimentos proprios (em ascensão recta e declinação)
    3 Distância
    4 Velocidade Radial

#### **APOGEE DATA**

In [1]:
from IPython.display import IFrame
IFrame(src='https://www.sdss.org/dr14/irspec/spectro_data/', width=1000, height=200)

#### **APOGEE DATA (Informação da tabela de dados)**

In [2]:
from IPython.display import IFrame
IFrame(src='https://data.sdss.org/datamodel/files/APOGEE_REDUX/APRED_VERS/APSTAR_VERS/ASPCAP_VERS/RESULTS_VERS/allStar.html#example', width=1000, height=200)

#### **Lendo o arquivo fits que contem os dados do APOGEE**

In [3]:
import pandas as pd
import numpy as np
import fitsio
import matplotlib.pyplot as plt

In [4]:
apogee_data_filename = 'allStar-l31c.2.fits'
column_name_apogee = ['APOGEE_ID','RA', 'DEC', 'VHELIO_AVG', 'PMRA', 'PMDEC', 'SNR'] # Selecting some columns from the original file
RAD = fitsio.read(apogee_data_filename, columns=column_name_apogee) # reading apogee data (RAD)
RAD = RAD.byteswap().newbyteorder()
APOGEE_TABLE_DF = pd.DataFrame.from_records(RAD) # Apogee Table as DataFrame

OSError: File not found: 'allStar-l31c.2.fits'

#### **Eliminando as filas quando o APOGEE_ID se repete mas mantendo a fila com o maior razão sinal ruido**

In [ ]:
# apogee data sorting by higuest SNR for duplicate APOGEE_ID
ADSBSNR = APOGEE_TABLE_DF.sort_values('SNR', ascending=False).drop_duplicates('APOGEE_ID').sort_index() 
ADSBSNR.head(10)
ADSBSNR.shape

#### **Guardando o novo arquivo em formato fits**
Com o fim de realizar o crossmatch com o catalogo GaiaDR2 será criado um novo arquivo com o nome de apogee_filter_SNR.fits

In [ ]:
result_file = fitsio.FITS('apogee_filter_SNR.fits','rw') 
result_file.write(ADSBSNR.to_records(index=False))
result_file.close()

#### **APOGEE e GAIADR2**
A fim de determinar a velocidade espacial em coordenadas galactocentricas é necessário conhecer as distâncias (não fornecido pelo apogee), portanto, os dados do apogee serão complementados com os dados do GAIADR2 que fornece o paralaxe por meio do qual é possível determinar a distância. Com essa finalidade iremos a fazer um crossmatch entre os dos levantamentos. A ferramenta para o crossmatch será feito com TOPCAT (*Tool for Operations on Catalogues And Tables*). Para realizar o crossmatch é necessário definir um radio de match, neste caso iremos usar o radio de 0.5 segundos de arco.

#### **Lendo o arquivo fits apos o crossmatch**
O crossmatch foi feito usando o catálogo contendo as distâncias calculadas a partir da paralaje como mostrado no artigo http://www.mpia.de/~calj/gdr2_distances/gdr2_distances.pdf, mas inicialmente foi realizado um crossmatch com o gaiaDR2 a fim de obter os movimentos proprios que não é dado no catalogo de distâncias localizado no vizier I/347/gaia2dis. O catalogo apos os crossmatch tem o nome APOGEExGAIADR2_dist.fits

In [ ]:
apogee_x_gaiadr2_distance_filename = 'APOGEExGAIADR2_dist.fits'
column_name_apogee_gaiadr2_distance = ['RA','DEC', 'PMRA', 'PMDEC','VHELIO_AVG', 'ra_epoch2000', 'dec_epoch2000', 'parallax', 'parallax_error', 'pmra_G', 'pmra_error', 'pmdec_G', 'pmdec_error', 'radial_velocity', 'radial_velocity_error', 'rest', 'b_rest_x', 'B_rest_xa']
AGD = fitsio.read(apogee_x_gaiadr2_distance_filename, ext=1, columns = column_name_apogee_gaiadr2_distance) # Apogee Gaia Distance (AGD)
AGD = AGD.byteswap().newbyteorder()
APO_GAIA_DIST = pd.DataFrame.from_records(AGD)

#### **Pasando as colunas do dataframe para arrays**

In [ ]:
RA = APO_GAIA_DIST.RA.values
DEC = APO_GAIA_DIST.DEC.values
PMRA = APO_GAIA_DIST.PMRA.values
PMDEC= APO_GAIA_DIST.PMDEC.values
VHELIO_AVG = APO_GAIA_DIST.VHELIO_AVG.values
rest = APO_GAIA_DIST.rest.values

In [ ]:
# Rodando o codigo mostrado no site do astropy 
c1 = coord.ICRS(ra=RA*u.degree, dec=DEC*u.degree,
                distance=(rest*u.pc),
                pm_ra_cosdec=PMRA*u.mas/u.yr,
                pm_dec=PMDEC*u.mas/u.yr,
                radial_velocity=VHELIO_AVG*u.km/u.s)
gc1 = c1.transform_to(coord.Galactocentric)
gc1.v_x[0]

### **Galactocentric coordinate**
Com todos os parâmetros necessários neste paso determinamos a velocidade en coordenadas galactocentricas, para isso iremos fazer uso da rutina em python do astropy. 

In [ ]:
from IPython.display import IFrame
IFrame(src='http://docs.astropy.org/en/stable/coordinates/index.html', width=1000, height=200)

In [ ]:
from IPython.display import IFrame
IFrame(src='http://docs.astropy.org/en/stable/coordinates/galactocentric.html#', width=1000, height=200)

In [ ]:
# Colocando os pacotes necessários para rodar o codigo
import numpy as np
import matplotlib.pyplot as plt
from astropy.visualization import astropy_mpl_style
plt.style.use(astropy_mpl_style)
# using astropy
import astropy.coordinates as coord
import astropy.units as u

In [ ]:
# Rodando o codigo mostrado no site do astropy 
c1 = coord.ICRS(ra=89.014303*u.degree, dec=13.924912*u.degree,
                distance=(37.59*u.mas).to(u.pc, u.parallax()),
                pm_ra_cosdec=372.72*u.mas/u.yr,
                pm_dec=-483.69*u.mas/u.yr,
                radial_velocity=0.37*u.km/u.s)

In [ ]:
gc1 = c1.transform_to(coord.Galactocentric)
print(gc1.v_x, gc1.v_y, gc1.v_z)

#### **Testando com os valores de velocidades galactocentricas obtidos para o Large Magellanic Cloud (LMC) e o Small Magellanic Cloud (SMC) no artigo doi:10.1088/0004-6256/137/5/4339** 
#### **Dados necessários para o LMC**
Posição 

1. RA  = +89.33 graus
2. DEC = -67.22 graus

Distância

3. distancia = 50100 pc

Movimento proprios

movimento proprio para velocidade de rotação  50 km/s

4. mu_alpha_cos_delta = +1.82 +- 0.13 mas/yr
5. mu_delta = +0.39 +- 0.15 mas/yr

movimento proprio para velocidade de rotação  120 km/s

6. mu_alpha_cos_delta = +1.61 +- 0.13 mas/yr
7. mu_delta = +0.60 +- 0.15 mas/yr

Velocidade radial 

8. VR = +287.0 km/s

In [ ]:
# velocidade galactocentrica para o LMC com velocidade de rotação de 50 km/s
C_LMC50_1 = coord.ICRS(ra=81.90*u.degree, dec=-69.87*u.degree,
                distance=50100*u.pc,
                pm_ra_cosdec=1.82*u.mas/u.yr,
                pm_dec=-0.39*u.mas/u.yr,
                radial_velocity=262.1*u.km/u.s)
gc_LMC50_1 = C_LMC50_1.transform_to(coord.Galactocentric)
print(gc_LMC50_1.v_x, gc_LMC50_1.v_y, gc_LMC50_1.v_z)

In [ ]:
# considerando os valores do artigo
v_sun = coord.CartesianDifferential([14.0, 12.24, 7.25]*u.km/u.s)
gc_frame_lmc50 = coord.Galactocentric(galcen_distance=8.2*u.kpc,
                                galcen_v_sun=v_sun,
                                z_sun=25*u.pc)
gc_LMC50_2 = C_LMC50_1.transform_to(gc_frame_lmc50)
print(gc_LMC50_2.v_x, gc_LMC50_2.v_y, gc_LMC50_2.v_z)

# Artigo Gaia DR2 in 6D: Searching for the fastest stars in the galaxy
Posição 

1. RA  = 45.13214374784831 graus
2. DEC = 0.13785096598403554 graus

Distância

3. distancia = 382.4321158364128 pc

Movimento proprios

movimento proprio para velocidade de rotação  50 km/s

4. mu_alpha_cos_delta = 4.782390524372778 mas/yr
5. mu_delta = 18.790211615251437 mas/yr

Velocidade radial 

8. VR = 33.84043726251102 km/s

In [ ]:
# velocidade galactocentrica para a primeira estrela do catalogo gaia (Marchetti)
GAIA_1 = coord.ICRS(ra=45.13214374784831*u.degree, dec=0.13785096598403554*u.degree,
                distance=382.4321158364128*u.pc,
                pm_ra_cosdec=4.782390524372778*u.mas/u.yr,
                pm_dec=18.790211615251437*u.mas/u.yr,
                radial_velocity=33.84043726251102*u.km/u.s)
gc_GAIA_1 = GAIA_1.transform_to(coord.Galactocentric)
print(gc_GAIA_1.v_x, gc_GAIA_1.v_y, gc_GAIA_1.v_z)

In [ ]:
# considerando os valores do artigo
v_sun = coord.CartesianDifferential([14.0, 250.24, 7.25]*u.km/u.s)
gc_frame = coord.Galactocentric(galcen_distance=8.2*u.kpc,
                                galcen_v_sun=v_sun,
                                z_sun=25*u.pc)
gc_GAIA_1C = GAIA_1.transform_to(gc_frame)
print(gc_GAIA_1C.v_x, gc_GAIA_1C.v_y, gc_GAIA_1C.v_z)

In [ ]:
# velocidade galactocentrica para a segunda estrela do catalogo gaia (Marchetti)
GAIA_2 = coord.ICRS(ra=45.05816745630085*u.degree, dec=0.12740379563087656*u.degree,
                distance=573.9735889753617*u.pc,
                pm_ra_cosdec=6.399015950035163*u.mas/u.yr,
                pm_dec=-12.94271166934367*u.mas/u.yr,
                radial_velocity=4.5726744498926255*u.km/u.s)
gc_GAIA_2 = GAIA_2.transform_to(gc_frame)
U = gc_GAIA_2.v_x
V = gc_GAIA_2.v_y
W = gc_GAIA_2.v_z
V_total = np.sqrt(U**2 + V**2 + W**2)
print(U, V, W, V_total)

In [ ]:
np.sqrt(17.877407021113946**2 + 212.54059105492033**2 + -4.06084412139914**2)